# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» предлагает новым пользователям на выбор два тарифа: «Смарт» или «Ультра». Однако было установлено, что старые клиенты до сих пор используют архивные тарифы. Чтобы способствовать переходу пользователей на новые тарифы, принято решение построить систему, способную проанализировать поведение клиентов и предложить им новый тариф.

В нашем распоряжении данные о поведении клиентов, которые уже перешли на тарифы «Смарт» и «Ультра». Обучим на них разные модели и определим наиболее точную из них.

**Цель исследования**: построить модель для задачи классификации, которая выберет подходящий тариф.

**Ход исследования**

Данные для исследования получим из файла `users_behavior.csv`. Известно, что предобработка данных не требуется.
 
Исследование пройдет в пять этапов:
 1. Изучение данных.
 2. Разбиение данных на выборки.
 3. Ислледование модели.
 4. Проверка модели на тестовой выборке.
 5. Проверка модели на адекватность.

## Изучение данных из файла

Для начала импортируем необходимые в исследовании библиотеки.

In [1]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier 

Прочитаем файл `users_behavior.csv` из каталога `datasets` и сохраним его в одноименной переменной.

In [2]:
users_behavior = pd.read_csv('/datasets/users_behavior.csv')

Составим представление о данных, отобразив на экране первые десять строк таблицы.

In [3]:
users_behavior.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


Получим общую информацию о таблице методом `info`.

In [4]:
users_behavior.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Итак, таблица `users_behavior` состоит из 5 столбцов:
* сalls — количество звонков,
* minutes — суммарная длительность звонков в минутах,
* messages — количество sms-сообщений,
* mb_used — израсходованный интернет-трафик в Мб,
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

### Выводы

В нашем распоряжении таблица `users_behavior`, содержащая данные о поведении пользователей. Согласно пояснению к данным, предобработка была проведена ранее, поэтому можем переходить к делению данных на выборки. 

Отметим, что, в таблице имеются схожие по смыслу столбцы: количество звонков и их суммарная длительность. Наличие в таблице сильно коррелируемых признаков может сказаться на качестве модели.

## Разбиение данных на выборки

Для начала проверим предположение о наличии сильной зависимости между значениями столбцов `calls` и `minutes` с помощью метода `corr()`.

In [5]:
print(f'Коэффициент корреляции между столбцами calls и minutes: '
      f'{users_behavior["calls"].corr(users_behavior["minutes"])}')

Коэффициент корреляции между столбцами calls и minutes: 0.9820832355742292


Коэффициент корреляции 0.98 говорит о наличии сильной связи между столбцами, что может привести к снижению производительности обобщения данных из-за высокой дисперсии и меньшей интерпретируемости модели. 

Целевым признаком нашей таблицы является столбец `is_ultra`: запишем его в переменную `target`. Влияющими признаками в нашем случае будут столбцы `minutes`, `messages` и `mb_used`. Столбец `calls` исключим из обучения нашей модели из-за сильной корреляции со столбцом `minutes`.

In [6]:
features = users_behavior.drop(['is_ultra', 'calls'], axis=1)
target = users_behavior['is_ultra']

Поделим исходный набор данных на обучающую, валидационную и тестовую выборки в отношении 2:1:1 с помощью метода `train_test_split()`: сначала применим его ко всему набору данных, отделив обучающую выборку, затем оставшийся набор данных разделим на валидационный и тестовый.

In [7]:
(features_train, features_valid_and_test, 
 target_train, target_valid_and_test) = (train_test_split
                                         (features, 
                                          target, 
                                          test_size=0.4, 
                                          random_state=123))
(features_valid, features_test, 
target_valid, target_test) = (train_test_split
                              (features_valid_and_test, 
                               target_valid_and_test, 
                               test_size=0.5, 
                               random_state=123))

Проверим размеры полученных выборок функцией `shape`.

In [8]:
print(features_train.shape, features_valid.shape, features_test.shape)

(1928, 3) (643, 3) (643, 3)


### Выводы

Разделив имеющиеся данные на выборки и определив влияющие и целевой признаки, перейдем к обучению модели. 

## Исследование модели

Для решения задачи классификации будем использовать три модели:
* дерево решений,
* случайный лес,
* логистическая регрессия.

Определим лучшее дерево решений, изменяя значения гиперпараметров `max_depth` (максимальная глубина дерева), `min_samples_split` (минимальное число объектов, необходимое для того, чтобы узел дерева мог расщепиться) и `min_samples_leaf` (минимальное число объектов в листьях).

In [9]:
best_model = None
best_result = 0
for depth in range(1, 5):
    for samples_split in (2, 30):
         for samples_leaf in (1, 30):
            model = (DecisionTreeClassifier(random_state=123, 
                                            max_depth=depth, 
                                            min_samples_split=samples_split, 
                                            min_samples_leaf=samples_leaf))
            model.fit(features_train, target_train)
            predictions = model.predict(features_valid)
            result = accuracy_score(target_valid, predictions)
            if result > best_result:
                best_model = model
                best_result = result
            
print(f'Лучшая модель на валидационной выборке: {best_model}')            
print(f'Accuracy наилучшей модели на валидационной выборке: {best_result}')

Лучшая модель на валидационной выборке: DecisionTreeClassifier(max_depth=4, min_samples_leaf=30, random_state=123)
Accuracy наилучшей модели на валидационной выборке: 0.8055987558320373


Лучшим деревом решений с `accuracy` = 0.806 является дерево со значением гиперпараметров `max_depth` = 4, `min_samples_leaf` = 30 и дефолтными `min_samples_split` = 2.

Найдем лучшие гиперпараметры для модели случайный лес.

In [10]:
best_model = None
best_result = 0
for estimators in range(1, 30):
    for depth in range(1, 5):
        for samples_split in (2, 30):
            for samples_leaf in (1, 30):
                model = (RandomForestClassifier(random_state=123, 
                                                n_estimators=estimators, 
                                                max_depth=depth, 
                                                min_samples_split=samples_split, 
                                                min_samples_leaf=samples_leaf))
                model.fit(features_train, target_train)
                predictions = model.predict(features_valid)
                result = accuracy_score(target_valid, predictions)
                if result > best_result:
                    best_model = model
                    best_result = result
            
print(f'Лучшая модель на валидационной выборке: {best_model}')            
print(f'Accuracy наилучшей модели на валидационной выборке: {best_result}')

Лучшая модель на валидационной выборке: RandomForestClassifier(max_depth=4, n_estimators=9, random_state=123)
Accuracy наилучшей модели на валидационной выборке: 0.8087091757387247


Случайный лес с гиперпараметрами `max_depth` = 4 и `n_estimators` = 9 позволил добиться лучшего `accuracy` = 0.809.

Для логистической регрессии лучшие гиперпараметры подберем методом `GridSearchCV()`.

In [11]:
param_grid = {'C': [0.001, 0.01, 0.05, 0.1, 0.5, 1.0, 10.0], 'penalty': ['l1', 'l2']}
clf = (GridSearchCV(LogisticRegression(random_state=12345, solver='liblinear'), 
                    param_grid, cv=5, scoring='accuracy'))
best_model = clf.fit(features_train.append(features_valid), target_train.append(target_valid))
print(f'Лучшая модель на валидационной выборке: {best_model.best_estimator_}')
print(f'Accuracy наилучшей модели на валидационной выборке: '
      f'{best_model.best_score_}')

Лучшая модель на валидационной выборке: LogisticRegression(C=10.0, penalty='l1', random_state=12345, solver='liblinear')
Accuracy наилучшей модели на валидационной выборке: 0.7526281591175248


Гиперпараметры логистической регрессии `C` = 10.0, `penalty` = l1, `solver` = liblinear помогли модели достичь значения `accuracy` = 0.753.

### Выводы

В качестве моделей обучения были выбраны дерево решений, случайный лес и логистическая регрессия. Изменяя значения гиперпараметров мы смогли достичь на валидационной выборке лучшего значения `accuracy`. Наиболее точной оказалась модель случайного леса, состоящая из 9 деревьев с глубиной дерева, равной 4 (`accuracy` = 0.809). Перейдем к проверке модели на тестовой выборке.

## Проверка модели на тестовой выборке

Обучим выбранную модель на обучающей и валидационной выборках, затем проверим ее на тестовой выборке.

In [12]:
model = RandomForestClassifier(max_depth=4, n_estimators=9, random_state=123)
model.fit(features_train.append(features_valid), target_train.append(target_valid))
predictions = model.predict(features_test)
result = accuracy_score(target_test, predictions)
print(f'{model}\nAccuracy: {result}\n')

RandomForestClassifier(max_depth=4, n_estimators=9, random_state=123)
Accuracy: 0.7916018662519441



### Выводы

Модель случайного леса, обученная на обучающей и валидационной выборках, дала значение `accuracy` = 0.792 на тестовой выборке, т.е. точность предсказания модели 79%.

## Проверка модели на адекватность.

Проверим наши модели на адекватность, сравнив значение их accuracy c полученным для константной модели. Если оно выше, то считаем, что модель адекватна.

In [13]:
model = DummyClassifier(strategy="most_frequent") 
model.fit(features_train, target_train) 
predictions = model.predict(features_valid)
result = accuracy_score(target_valid, predictions)
print(f'Accuracy константной модели на валидационной выборке: {result}')

Accuracy константной модели на валидационной выборке: 0.6982892690513219


### Выводы

Значение accuracy константной модели на валидационной выборке оказалось ниже значений обученных ранее моделей. Таким образом, можно считать, что наши модели обучения являются адекватными.

## Общий вывод

На основе данных поведения клиентов, использующих тарифы "Смарт" и "Ультра" были проведено обучение трех моделей: дерево решений, случайный лес и логистическая регрессия для подбора пользователю подходящего тарифа. 

Для каждой модели были определены гиперпараметры, позволяющие добиться лучшего значения метрики `accuracy`. Для проверки тестовой выборки был выбран случайный лес, состоящий из 9 деревьев с глубиной дерева, равной 4. На тестовой выборке модель обучения достигла значения `accuracy` = 0.792. 

Дополнительно обученные модели были проверены на адекватность путем сравнения полученного `accuracy` со значением для константной модели.